In [ ]:
!pip install pygit2==1.15.1
%cd /content
!git clone https://github.com/lllyasviel/Fooocus.git
%cd /content/Fooocus


In [ ]:
#@title #**Descargar modelo y actualizar presets**
from IPython.display import display, HTML
import os, json, requests
%cd /content/Fooocus
model_url = input("🔗 Pega aquí el enlace de Hugging Face o Civitai (.safetensors): ").strip()
token_civitai = "" # @param {"type":"string"}
model_dir = 'models/checkpoints'
default_json_path = 'presets/default.json'
os.makedirs(model_dir, exist_ok=True)
filename = model_url.split('/')[-1]
model_path = os.path.join(model_dir, filename)
download_url = f'{model_url}?token={token_civitai}' if token_civitai else model_url
if not os.path.exists(model_path):
    display(HTML("<h1 style='color: yellow;'>Descargando modelo por favor espere...</h1>"))
    r = requests.get(download_url, stream=True)
    with open(model_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    display(HTML("<h1 style='color: cyan;'>Descarga Finalizada</h1>"))
else:
    print('⚠️ El modelo ya existe. No se descargó de nuevo.')
with open(default_json_path, 'r') as f:
    data = json.load(f)
data['default_model'] = filename
data.setdefault('checkpoint_downloads', {})[filename] = model_url
if filename in data.get('previous_default_models', []):
    data['previous_default_models'].remove(filename)
with open(default_json_path, 'w') as f:
    json.dump(data, f, indent=4)
print(f'✅ Se actualizó "default_model" a: {filename}')
print(f'📁 Modelo listo para usar en la carpeta: {model_path}')


In [ ]:
#@title #**Descargar Loras (Civitai o Hugginface)**
#@markdown Ingresa tu token de civitai o dejalo en blanco si es un link de hugginface. <p>
#@markdown separa los links por comas `(,)`
from IPython.display import display, HTML
from IPython.display import clear_output
%cd /content/Fooocus/models/loras
lora_urls = "" # @param {"type":"string"}

token_civitai = "" # @param {"type":"string"}

# Separa las URLs por comas
urls = [url.strip() for url in lora_urls.split(',') if url.strip()] # Ignora URLs vacías

# Descarga cada archivo
for url in urls:
    if token_civitai:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={token_civitai}
    else:
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

# Muestra mensaje de descarga finalizada
display(HTML("<h1 style='color: cyan;'>Descarga Finalizada</h1>"))

In [ ]:
#@title #**Inicia la webui**
#@markdown **INICIA FUTURE FOOCUS**
import os
from IPython.display import clear_output
# parche para ram
if 'installed' not in globals():
    !sudo apt-get install -y libjemalloc-dev -q
    !gdown --fuzzy https://drive.google.com/file/d/1NFABbKFE19HpK1mqUJpFjFefY_oA9phm/view?usp=sharing -O /content/libjemalloc.so.2
    installed = True  # Marcar como instalado
else:
    print("\033[93mEL parche de ram ya se ha instalado. No es necesario reinstalar.\033[0m")
    print("\033[93mSaltando...\033[0m")

%cd /content/Fooocus
!uv pip install -r requirements_versions.txt --no-progress
!uv pip install xformers==0.0.29.post3 -f https://download.pytorch.org/whl/cu124 --no-progress
!python entry_with_update.py --share --always-high-vram
